# Transcription factor classifier

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from itertools import product
from nn import nn, io, preprocess

## Data

In [2]:
rap1 = io.read_text_file("./data/rap1-lieb-positives.txt")
yeast = io.read_fasta_file("./data/yeast-upstream-1k-negative.fa")

print("Length of Rap1 positives: " + str(len(rap1)))
print("Length of Yeast negatives: " + str(len(yeast)))

Length of Rap1 positives: 137
Length of Yeast negatives: 3163


In [3]:
pos_seq = rap1

# Break up yeast_neg into sizes match the length of rap1 sequences
seq_len = len(rap1[0])
neg_seq = []

for seq in yeast:
    seq_sub = [seq[i:i+seq_len] for i in range(0, len(seq), seq_len)]
    # Keep only sequences that are exactly rap length long
    seq_sub = [x for x in seq_sub if len(x) == seq_len]
    neg_seq += seq_sub

# Combine all sequences and get labes
seqs = pos_seq + neg_seq
labels = [True] * len(pos_seq) + [False] * len(neg_seq)

print("Length of positives: " + str(len(pos_seq)))
print("Length of negatives: " + str(len(neg_seq)))
print("Total sequences: " + str(len(pos_seq) + len(neg_seq)))

Length of positives: 137
Length of negatives: 183297
Total sequences: 183434


In [4]:
# Up sample the positive class
seqs2, labels2 = preprocess.sample_seqs(seqs, labels)
print("Length of positives: " + str(sum(labels2)))
print("Length of negatives: " + str(len(seqs2) - sum(labels2)))
print("Total sequences: " + str(len(seqs2)))

Length of positives: 183297
Length of negatives: 183297
Total sequences: 366594


In [5]:
# Encode sequences and create a training and testing split
X = preprocess.one_hot_encode_seqs(seqs2)
y = np.array(labels2)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 1)
print("Training data " + str(X_train.shape))
print("Testing data " + str(X_val.shape))

Training data (256615, 68)
Testing data (109979, 68)


For this dataset, I chose to upsample the positive class to match the number of observations in the negative class. The primary reason for this is because neural networks tend to do better with a larger number of observations and having 274 total observations would not yield a productive neural network function. Also 

# 3-fold cross validation of tuning of neural network layers
Parameters to test:
* Number of batches: 100 vs 100000
* Number of epochs: 10 vs 500
* Learning rate: 0.01 vs 0.0001

For the layers of the neural network, since we are trying to reduce the data from it's 68 features to a classification of True vs False, I chose to use a three layer neural network, halving the number of parameters till I arbitrarily felt like I didn't want to anymore. Since we are trying to perform a binary classification, a sigmoid activation seems most appropriate. 

In [6]:
num_cv = 3 # number of cross validation folds
# Create dataframe to store training results
cols = ["idx", "train_error", "val_error",]
tune_res = pd.DataFrame(columns = cols)

# Split the training set into three folds
kf = KFold(n_splits = num_cv)

#Define the parameters
layers = [{"input_dim" : 68, "output_dim" : 34, "activation" : "sigmoid"},
          {"input_dim" : 34, "output_dim" : 17, "activation" : "sigmoid"},
          {"input_dim" : 17, "output_dim" : 8, "activation" : "sigmoid"},
          {"input_dim" : 8, "output_dim" : 1, "activation" : "sigmoid"}]
num_batches = {100: 100, 
               100000: 100000}
num_epochs = {100: 100,
              500: 500}
num_lr = {0.01: 0.01,
          0.0001: 0.0001}

# Create a grid of parameter iterations
d = {'batches':[100, 100000], 
     'epochs':[100, 500],
     'lr':[0.01, 0.0001]}
param_grid = [dict(zip(d, v)) for v in product(*d.values())]
param_grid

[{'batches': 100, 'epochs': 100, 'lr': 0.01},
 {'batches': 100, 'epochs': 100, 'lr': 0.0001},
 {'batches': 100, 'epochs': 500, 'lr': 0.01},
 {'batches': 100, 'epochs': 500, 'lr': 0.0001},
 {'batches': 100000, 'epochs': 100, 'lr': 0.01},
 {'batches': 100000, 'epochs': 100, 'lr': 0.0001},
 {'batches': 100000, 'epochs': 500, 'lr': 0.01},
 {'batches': 100000, 'epochs': 500, 'lr': 0.0001}]

In [ ]:
# Iterate through the parameter grid
for idx, _ in enumerate(param_grid):
    # Get current parameters
    b = param_grid[idx]['batches']
    e = param_grid[idx]['epochs']
    lr = param_grid[idx]['lr']
    print("Testing parameter set " + str(idx))
    print("Batches = " + str(b) + ", Epochs = " + str(e) + ", lr = " + str(lr))
    
    fold_train_loss = []
    fold_val_loss = []
    
    tune_res.loc[idx, "idx"] = idx
    for cv_fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train)):
        
        net = nn.NeuralNetwork(nn_arch = layers,
                               batch_size = num_batches[b],
                               epochs = num_epochs[e],
                               lr = num_lr[lr],
                               seed = 1,
                               loss_function = "bce",
                               verbose = False)
        # Get fold-specific training and testing data
        train_loss, val_loss = net.fit(X_train[train_idx], y_train[train_idx],
                                       X_train[val_idx], y_train[val_idx])
        
        # Calculate average loss
        fold_train_loss.append(np.mean(train_loss))
        fold_val_loss.append(np.mean(val_loss))
    
    tune_res.loc[idx, "train_error"] = np.mean(fold_train_loss)
    tune_res.loc[idx, "val_error"] = np.mean(fold_val_loss)

Testing parameter set 0
Batches = 100, Epochs = 100, lr = 0.01


In [ ]:
# Results of tuning
tune_res

In [ ]:
# Get parameters of the tuning with the lowest validation error
opt_idx = tune_res["val_error"].astype(float).idxmin()
# Print parameters
print("Lowest validation with the following parameters:")
print("Batches: " + str(param_grid[opt_idx]['batches']))
print("Epochs: " + str(param_grid[opt_idx]['epochs']))
print("Learning rate: " + str(param_grid[opt_idx]['lr']))

## Run model with tuned parameters
Given the outputs of the tuning, run with the "optimal parameters", training on the full dataset and validating the test dataset.

In [ ]:
net = nn.NeuralNetwork(nn_arch = layers,
                       batch_size = num_batches[param_grid[opt_idx]['batches']],
                       epochs = num_epochs[param_grid[opt_idx]['epochs']],
                       lr = num_lr[param_grid[opt_idx]['lr']],
                       seed = 1,
                       loss_function = "bce",
                       verbose = False)

train_loss, val_loss = net.fit(X_train, y_train, X_val, y_val)

Plot the training and validation losses

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10,5))
fig.suptitle('Loss History')
axs[0].plot(np.arange(len(train_loss)), train_loss)
axs[0].set_title('Training Loss')
axs[0].set_xlabel("Epochs")
axs[0].set_ylabel('Loss')
axs[1].plot(np.arange(len(val_loss)), val_loss)
axs[1].set_title('Validation Loss')
axs[1].set_xlabel("Epochs")
axs[1].set_ylabel('Loss')
fig.tight_layout()
plt.show() 